### Imports

In [1]:
from simcse import SimCSE
import pandas as pd
import numpy as np
from pathlib import Path

### Load model and dataset

In [2]:
model = SimCSE('./data/sts_model/')

Some weights of BertModel were not initialized from the model checkpoint at ./data/dental_model/ and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
dataset = pd.read_csv('data/input/sts_simcse_test_input.csv')

### Process

In [4]:
sentence1_values = [str(i) for i in list(dataset['sentence1'].values)]
sentence2_values = [str(i) for i in list(dataset['sentence2'].values)]

In [5]:
sentence1_embedding = model.encode(sentence1_values)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:01<00:00, 15.59it/s]


In [6]:
sentence2_embedding = model.encode(sentence2_values)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 38.38it/s]


In [7]:
len(sentence2_embedding[0])

768

In [9]:
dataset['score']

0       2.5
1       3.6
2       5.0
3       4.2
4       1.5
       ... 
1374    0.0
1375    1.0
1376    1.0
1377    0.0
1378    0.0
Name: score, Length: 1379, dtype: float64

In [13]:
sts_clutser_embeddings = []

# Loop both embeddings and enter them in the new array
for i in range(dataset.shape[0]):
    sts_clutser_embeddings.append([
        i,
        dataset['score'][i],
        1,
        np.array(sentence1_embedding[i])])
    
    sts_clutser_embeddings.append([
        i,
        dataset['score'][i],
        2,
        np.array(sentence2_embedding[i])])

In [14]:
sts_cluster_input = pd.DataFrame(sts_clutser_embeddings)
sts_cluster_input.columns = ['row','score','sentence number','sentence embedding']

In [15]:
sts_cluster_input

,row,score,sentence number,sentence embedding
0,0,2.5,1,"[0.028135695, 0.07861127, -0.004314823, -0.028..."
1,0,2.5,2,"[0.018967861, 0.06663294, -0.05224602, -0.0317..."
2,1,3.6,1,"[-0.008153794, 0.0068986965, -0.084251024, 0.0..."
3,1,3.6,2,"[-0.0041697943, 0.0023616785, -0.07975078, 0.0..."
4,2,5.0,1,"[-0.0062140888, 0.041082907, -0.021769198, -0...."
...,...,...,...,...
2753,1376,1.0,2,"[0.011161995, 0.01629606, -0.026892785, 0.0733..."
2754,1377,0.0,1,"[-0.04969108, -0.0004373235, 0.027179077, 0.08..."
2755,1377,0.0,2,"[0.00799096, -0.054691225, 0.02732386, 0.03177..."
2756,1378,0.0,1,"[-0.008215942, 0.001497228, -0.004740279, -0.0..."


In [16]:
# Check folder exists
Path('./data/output').mkdir(parents=True, exist_ok=True)

# Export pandas dataframe to .csv
sts_cluster_input.to_csv('data/output/sts_cluster_input.csv')